## Tweet analytics

# Google Collab With TPU 

In [0]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
# !tar xf spark-2.4.4-bin-hadoop2.7.tgz
# !pip install -q findspark

In [0]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
# import findspark
# findspark.init()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# importing all libraries

In [5]:
import tweepy
from tweepy import OAuthHandler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import re
import nltk
from nltk import tokenize
from collections import defaultdict
import pandas as pd
import numpy as np
import pickle
import sys
import os
import io
import re
from sys import path
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from string import punctuation, digits
from IPython.core.display import display, HTML
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

import nltk
nltk.download('stopwords')
# from pyspark.sql import Row
# from pyspark.sql import SQLContext
# from pyspark import SparkFiles
# from pyspark.ml.linalg import DenseVector
from sklearn.metrics import accuracy_score
import pandas as pd


# import pyspark
# from pyspark import SparkContext


import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import train_test_split
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report,recall_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier



from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.semi_supervised import LabelSpreading
from sklearn.svm import LinearSVC
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

nltk.download('wordnet')
from sklearn.metrics import roc_curve,precision_recall_curve
from sklearn.metrics import roc_auc_score,confusion_matrix

# from pyspark.sql import SparkSession


# from pyspark.ml.feature import HashingTF, IDF, Tokenizer
# from pyspark.ml.feature import StringIndexer
# from pyspark.ml import Pipeline


from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.semi_supervised import LabelSpreading
from sklearn.svm import LinearSVC
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
#Loading Data

In [0]:
df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/twitter_training.csv",encoding='latin',header=None)
df=df[[0,5]]
df=df.rename(columns={0: "Class", 5: "Tweet"})

Converting Postive sentiments which were denoted by 4 with 1

In [0]:
df.Class=df.Class.replace(4,1)


In [9]:
len(df)

1600000

In [10]:
df

,Class,Tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


# converting all the text data to lower letter and remove links

In [0]:
df['Lower']=df['Tweet'].str.lower()
df['Lower']=df['Lower'].str.replace('\d+', '')
df['Lower']=df['Lower'].str.replace(r"http\S+", '')



# removing stop words

In [0]:
stop_words = set(stopwords.words('english')) 
df['tweet_without_stopwords'] = df['Lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


# Stemming and Lemmatisation

In [0]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# Use English stemmer.
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()

df['lemm'] = df['tweet_without_stopwords'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
df['stemmed'] = df['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))

# Keeping only english letters

In [0]:
df['cleaned_specials'] = df['stemmed'].str.replace('[^A-z]', '')

In [15]:
df2=df
df.columns

Index(['Class', 'Tweet', 'Lower', 'tweet_without_stopwords', 'lemm', 'stemmed',
       'cleaned_specials'],
      dtype='object')

# Dropping all the intermediate column

In [16]:
df=df.drop(columns=['Lower','tweet_without_stopwords','lemm','stemmed','Tweet'])
df=df.rename(columns={'cleaned_specials': "Tweet"})
df


,Class,Tweet
0,0,switchfoot awww that bummer shoulda got david...
1,0,upset cant updat facebook text it might cri re...
2,0,kenichan dive mani time ball manag save rest ...
3,0,whole bodi feel itchi like fire
4,0,nationwideclass no behav all im mad here cant ...
...,...,...
1599995,1,woke up school best feel ever
1599996,1,thewdbcom cool hear old walt interviews
1599997,1,readi mojo makeover ask detail
1599998,1,happi th birthday boo alll time tupac amaru sh...


# Creating TFID Vector

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')


In [18]:
vectorizer.fit(df['Tweet'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
vector = vectorizer.transform(df['Tweet'])



# Pickle the vectorizer for model to be used in the application

In [20]:
from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(vectorizer, 'abc.pkl')
# vector

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['abc.pkl']

# Splitting in Train and Test in 70:30 ratio

In [21]:
from sklearn.model_selection import train_test_split

# splitting dataset into training and test data
trainX, testX = train_test_split(vector, test_size=0.30, random_state=1)
trainY, testY = train_test_split(df['Class'], test_size=0.30, random_state=1)
trainX

xTrain, xTest = train_test_split(vector, test_size=0.30, random_state=1)
yTrain, yTest = train_test_split(df['Class'], test_size=0.30, random_state=1)
trainX



<1120000x714322 sparse matrix of type '<class 'numpy.float64'>'
	with 7301901 stored elements in Compressed Sparse Row format>

# Applying Multinomial NB Model

In [22]:
from sklearn.naive_bayes import MultinomialNB

# Fitting Naibe Bayes to the training set
classifier = MultinomialNB().fit(trainX ,trainY)
from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(MultinomialNB, 'nb_Down.pkl')

['nb_Down.pkl']

In [0]:
all_predictions = classifier.predict(testX)

In [24]:
from sklearn.metrics import classification_report

# Calculating and printing the model accuracy and f1 score
print (classification_report(testY, all_predictions))

              precision    recall  f1-score   support

           0       0.74      0.78      0.76    239996
           1       0.77      0.73      0.75    240004

    accuracy                           0.75    480000
   macro avg       0.75      0.75      0.75    480000
weighted avg       0.75      0.75      0.75    480000



# Applying Logistic Regression Classifier and pickling it

In [25]:
from sklearn.linear_model import LogisticRegression

# Fitting LR to the training set
classifier2 = LogisticRegression(random_state=1)
classifier2.fit(trainX, trainY)

#Predicting the test set results
y_pred = classifier2.predict(testX)

# Calculating and printing the model accuracy and f1 score
print (classification_report(testY, y_pred))


from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(classifier2, 'lr_Down.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.79      0.74      0.76    239996
           1       0.76      0.80      0.78    240004

    accuracy                           0.77    480000
   macro avg       0.77      0.77      0.77    480000
weighted avg       0.77      0.77      0.77    480000



['lr_Down.pkl']

# Applying SGDC Classifier and pickling it

In [26]:
from sklearn.linear_model import SGDClassifier

# Fitting Stochastic Gradient Descent to the training set
classifier3 = SGDClassifier(random_state=1)
classifier3.fit(trainX, trainY)

#Predicting the test set results
y_prediction = classifier3.predict(testX)

# Calculating and printing the model accuracy and f1 score
print (classification_report(testY, y_prediction))

from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(classifier3, 'sgdc_Down.pkl')

              precision    recall  f1-score   support

           0       0.80      0.64      0.71    239996
           1       0.70      0.84      0.77    240004

    accuracy                           0.74    480000
   macro avg       0.75      0.74      0.74    480000
weighted avg       0.75      0.74      0.74    480000



['sgdc_Down.pkl']

# MaxAbs Scalar to scale the data ,as normal data was huge for SVC

In [0]:
from sklearn.preprocessing import MaxAbsScaler
scaling=MaxAbsScaler().fit(xTrain)
xTrain=scaling.transform(xTrain)
xTest=scaling.transform(xTest)

# Applying Linear SVC Model and pickling it

In [28]:
from sklearn.svm import LinearSVC
classifier3 = LinearSVC()
classifier3.fit(xTrain, yTrain)

from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(classifier3, 'linearsvc_Down.pkl')

['linearsvc_Down.pkl']

In [29]:
y_prediction = classifier3.predict(xTest)
# Calculating and printing the model accuracy and f1 score
print (classification_report(testY, y_prediction))
# SVC()

              precision    recall  f1-score   support

           0       0.76      0.74      0.75    239996
           1       0.75      0.77      0.76    240004

    accuracy                           0.76    480000
   macro avg       0.76      0.76      0.76    480000
weighted avg       0.76      0.76      0.76    480000



# Applying Random Forest Classifer and pickling it

In [30]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_jobs=-1,max_depth=3,random_state=1,warm_start=True)
rf_classifier.fit(xTrain,yTrain)
from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(rf_classifier, 'rf_Down.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


['rf_Down.pkl']

In [31]:
model2 = rf_classifier.predict(xTest)
#print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print (classification_report(testY, model2))

rfAccuracy=accuracy_score(yTest, model2)*100

The accuracy :-  51.90833333333333
              precision    recall  f1-score   support

           0       0.71      0.06      0.12    239996
           1       0.51      0.97      0.67    240004

    accuracy                           0.52    480000
   macro avg       0.61      0.52      0.39    480000
weighted avg       0.61      0.52      0.39    480000



In [0]:
# model=tree.DecisionTreeClassifier(criterion="entropy",max_depth=3,random_state=1)
# model.fit(xTrain,yTrain)
# from sklearn.externals import joblib 
# # Save the model as a pickle in a file 
# joblib.dump(model, 'rf_Down.pkl')

In [0]:
# model2 = model.predict(xTest)
# print(classification_report(yTest, model2))


In [0]:
# jhvakcvlb a

In [0]:
# neigh = KNeighborsClassifier(n_neighbors=2,n_jobs=-1,leaf_size=400)
# neigh.fit(xTrain, yTrain)


In [0]:
# neigh2 = neigh.predict(xTest)
# # #print(classification_report(yTest, neigh2))
# print('The accuracy :- ',accuracy_score(yTest, neigh2)*100)

# knnAccuracy=accuracy_score(yTest, model2)*100
